<a href="https://colab.research.google.com/github/Miit-009/Jio-Institute/blob/main/Session_8_(10_FEB)_Multi_Touch_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import permutations

# Step 1: Load the data
file_path = 'multi_touch_attribution_data.xlsx'
df = pd.read_excel(file_path)

# Step 2: Calculate Last Attribution and First Attribution
df['last_attribution'] = df['journey'].apply(lambda x: eval(x)[-1] if len(eval(x)) > 0 else None)
df['first_attribution'] = df['journey'].apply(lambda x: eval(x)[0] if len(eval(x)) > 0 else None)

# Step 3: Calculate Shapley Multitouch Attribution (Simplified version)
def calculate_shapley_value(journey, touchpoints_column):
    # Number of total possible touchpoints
    n = len(touchpoints_column)

    # Convert journey to list of touchpoints
    journey_list = eval(journey)

    # Shapley values storage
    shapley_values = {tp: 0 for tp in touchpoints_column}

    # Generate all permutations of the journey
    perms = list(permutations(journey_list))

    # Calculate marginal contribution for each touchpoint in each permutation
    for perm in perms:
        for i, touchpoint in enumerate(perm):
            prev_contribution = set(perm[:i])
            marginal_contribution = len(prev_contribution.intersection(set(touchpoints_column))) - len(prev_contribution.intersection(set(touchpoints_column)))
            shapley_values[touchpoint] += marginal_contribution

    # Normalize the Shapley values
    total_permutations = len(perms)
    for tp in shapley_values:
        shapley_values[tp] /= total_permutations

    return shapley_values

# Step 4: Apply Shapley attribution calculation
# Assuming 'journey' column contains the touchpoints and the other columns represent their contributions
touchpoints_columns = ['Email', 'Social Media', 'Search', 'Display', 'Referral', 'Direct']
df['shapley_values'] = df['journey'].apply(lambda journey: calculate_shapley_value(journey, touchpoints_columns))

# Step 5: Extract and visualize the results

# Visualize First Attribution
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='first_attribution')
plt.title('First Attribution Count')
plt.xticks(rotation=45)
plt.show()

# Visualize Last Attribution
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='last_attribution')
plt.title('Last Attribution Count')
plt.xticks(rotation=45)
plt.show()

# Visualize Shapley Attribution for each touchpoint
# Aggregating Shapley values for all rows
shapley_agg = pd.DataFrame(df['shapley_values'].tolist(), index=df.index)
shapley_agg = shapley_agg.mean()

# Plotting Shapley Values
plt.figure(figsize=(10, 6))
sns.barplot(x=shapley_agg.index, y=shapley_agg.values)
plt.title('Shapley Multitouch Attribution (Mean Values)')
plt.xticks(rotation=45)
plt.show()


In [ ]:
import pandas as pd
from collections import defaultdict
from itertools import chain, combinations
import numpy as np
import matplotlib.pyplot as plt

def powerset(iterable):
    """Returns the powerset of an iterable."""
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def shapley_value(conversions, channels):
    """Calculate Shapley values for each marketing channel."""
    channel_values = defaultdict(float)
    num_channels = len(channels)

    for channel in channels:
        for subset in powerset(channels - {channel}):
            subset = set(subset)
            subset_value = conversions.get(frozenset(subset), 0)
            larger_subset = subset | {channel}
            larger_value = conversions.get(frozenset(larger_subset), 0)
            marginal_contribution = larger_value - subset_value
            channel_values[channel] += marginal_contribution / (num_channels * len(list(powerset(subset))))

    return channel_values

def first_touch_attribution(data):
    """Calculate First-Touch Attribution."""
    first_touch = defaultdict(float)
    for journey, converted in data:
        if converted:
            first_touch[journey[0]] += 1
    return first_touch

def last_touch_attribution(data):
    """Calculate Last-Touch Attribution."""
    last_touch = defaultdict(float)
    for journey, converted in data:
        if converted:
            last_touch[journey[-1]] += 1
    return last_touch

# Upload and read CSV file
def load_csv(file_path):
    df = pd.read_csv(file_path)
    df['journey'] = df['journey'].apply(lambda x: eval(x))  # Convert string representation to list
    data = [(tuple(journey), converted) for journey, converted in zip(df['journey'], df['converted'])]
    return data

file_path = "/content/multi_touch_attribution_data.csv"  # Replace with the actual file path
data = load_csv(file_path)

# First Touch Attribution
first_touch_results = first_touch_attribution(data)
print("First Touch Attribution:", first_touch_results)

# Last Touch Attribution
last_touch_results = last_touch_attribution(data)
print("Last Touch Attribution:", last_touch_results)

# Shapley Value Attribution
conversions = defaultdict(int)
for journey, converted in data:
    if converted:
        for subset in powerset(journey):
            conversions[frozenset(subset)] += 1

channels = set(chain(*[journey for journey, _ in data]))
shapley_results = shapley_value(conversions, channels)
print("Shapley Value Attribution:", shapley_results)

# Plot results
def plot_results(results, title):
    plt.figure(figsize=(10, 5))
    plt.bar(results.keys(), results.values(), color='skyblue')
    plt.xlabel("Channels")
    plt.ylabel("Attribution Value")
    plt.title(title)
    plt.xticks(rotation=45)
    plt.show()

plot_results(first_touch_results, "First Touch Attribution")
plot_results(last_touch_results, "Last Touch Attribution")
plot_results(shapley_results, "Shapley Value Attribution")